In [3]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
!pip install transformers==4.4.0
!pip install sklearn

In [5]:
!git clone https://github.com/declare-lab/MELD.git

fatal: 대상 경로가('MELD') 이미 있고 빈 디렉터리가 아닙니다.


In [6]:
import glob
data_path = "./MELD/data/MELD/*.csv"
data_path_list = glob.glob(data_path)
print(data_path_list)

['./MELD/data/MELD/train_sent_emo.csv', './MELD/data/MELD/test_sent_emo.csv', './MELD/data/MELD/dev_sent_emo.csv']


In [7]:
import csv


# 데이터 저장
def split(session):
    ret_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        ret_data.append(split_session[:])
    return ret_data

for data_path in data_path_list:
    f = open(data_path, 'r')
    rdr = csv.reader(f)

    # 세션 데이터 저장할 리스트
    session_dataset = []
    session = []
    speaker_set = []

    # 실제 데이터 저장 방식
    pre_session = 'start'
    for i, line in enumerate(rdr):
        if i == 0:
            # 저장할 데이터들 index 확인
            header = line
            utt_idx = header.index('Utterance')
            speaker_idx = header.index('Speaker')
            emo_idx = header.index('Emotion')
            session_idx = header.index('Dialogue_ID')
        else:
            utt = line[utt_idx]
            speaker = line[speaker_idx]

            # Unique한 스피커로 바꾸기
            if speaker in speaker_set:
                uniq_speaker = speaker_set.index(speaker)
            else:
                speaker_set.append(speaker)
                uniq_speaker = speaker_set.index(speaker)
            emotion = line[emo_idx]
            sess = line[session_idx]

            if pre_session == 'start' or sess == pre_session:
                session.append([uniq_speaker, utt, emotion])
            else:
                # 세션 데이터 저장
                # session_dataset.append(session)
                session_dataset += split(session)
                session = [[uniq_speaker, utt, emotion]]
                speaker_set = []
            pre_session = sess

    # 마지막 세션 저장
    session_dataset += split(session)
    f.close()

In [8]:
session_dataset

[[[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness']],
 [[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness'],
  [1, 'What?', 'surprise']],
 [[2,
   'Or! Or, we could go to the bank, close our accounts and cut them off at the source.',
   'neutral']],
 [[2,
   'Or! Or, we could go to the bank, close our accounts and cut them off at the source.',
   'neutral'],
  [0, 'You’re a genius!', 'joy']],
 [[2,
   'Or! Or, we could go to the bank, close our accounts and cut them off at the source.',
   'neutral'],
  [0, 'You’re a genius!', 'joy'],
  [1, 'Aww, man, now we won’t be bank buddies!', 'sadness']],
 [[2,
   'Or! Or, we could go to the bank, close our accounts and cut them off at the source.',
   'neutral'],
  [0, 'You’re a genius!', 'joy'],
  [1, 'Aww, man, now we won’t be bank buddies!', 'sadness'],
  [0, 'Now, there’s two reasons.', 'neutral']],
 [[2,
   'Or! Or, we could go to the bank, close our accounts and cut them off at the source.',
   'neutral'],
  [0

In [9]:
from torch.utils.data import Dataset

class data_loader(Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        rdr = csv.reader(f)

        # 추가할 때, 중복되지 않도록 set 사용
        emoSet = set()

        # 세션 데이터 저장할 것
        self.session_dataset = []
        session = []
        speaker_set = []

        # 실제 데이터 저장 방식
        pre_sess = 'start'
        for i, line in enumerate(rdr):
            if i == 0:
                # 저장할 데이터들 index 확인
                header  = line
                utt_idx = header.index('Utterance')
                speaker_idx = header.index('Speaker')
                emo_idx = header.index('Emotion')
                sess_idx = header.index('Dialogue_ID')
            else:
                utt = line[utt_idx]
                speaker = line[speaker_idx]

                if speaker in speaker_set:
                    uniq_speaker = speaker_set.index(speaker)
                else:
                    speaker_set.append(speaker)
                    uniq_speaker = speaker_set.index(speaker)
                emotion = line[emo_idx]
                sess = line[sess_idx]

                if pre_sess == 'start' or sess == pre_sess:
                    session.append([uniq_speaker, utt, emotion])
                else:
                    self.session_dataset += split(session)
                    session = [[uniq_speaker, utt, emotion]]
                    speaker_set = []
                    emoSet.add(emotion)
                pre_sess = sess
        self.session_dataset += split(session)

        self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
        self.emoList = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
        f.close()

    def __len__(self):
        return len(self.session_dataset)

    def __getitem__(self, idx):
        return self.session_dataset[idx]

    # batch process
    def collate_fn(self, sessions):
        '''
            input:
                data: [(session1), (session2), ... ]
            return:
                batch_input_tokens_pad: (B, L) padded
                batch_labels: (B)
        '''
        batch_input_token = []
        for session in sessions:
            input_token = ""
            for line in session:
                speaker, utt, emotion = line
                input_token += utt
            batch_input_token.append(input_token)

        return batch_input_token

In [10]:
dev_dataset = data_loader("./MELD/data/MELD/dev_sent_emo.csv")

In [11]:
dev_dataset[0], dev_dataset[1]

([[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness']],
 [[0, 'Oh my God, he’s lost it. He’s totally lost it.', 'sadness'],
  [1, 'What?', 'surprise']])

In [12]:
# 배치 결과 확인
from torch.utils.data import DataLoader
dev_dataloader = DataLoader(dev_dataset, batch_size=4, shuffle=False, num_workers=4, collate_fn=dev_dataset.collate_fn)

In [13]:
dev_dataloader

In [14]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [15]:
dir(tokenizer)
tokenizer.model_max_length

512

In [16]:
# tokenizer
res = tokenizer('hello, my name is minseong')
print(res)

{'input_ids': [0, 42891, 6, 127, 766, 16, 5251, 1090, 1657, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [17]:
def split(session):
    final_data = []
    split_session = []
    for line in session:
        split_session.append(line)
        final_data.append(split_session[:])
    return final_data

class data_loader(Dataset):
    def __init__(self, data_path):
        f = open(data_path, 'r')
        rdr = csv.reader(f)

        emoSet = set()
        self.tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

        self.session_dataset = []
        session = []
        speaker_set = []

        pre_sess = 'start'
        for i, line in enumerate(rdr):
            if i == 0:
                header  = line
                utt_idx = header.index('Utterance')
                speaker_idx = header.index('Speaker')
                emo_idx = header.index('Emotion')
                sess_idx = header.index('Dialogue_ID')
            else:
                utt = line[utt_idx]
                speaker = line[speaker_idx]
                if speaker in speaker_set:
                    uniq_speaker = speaker_set.index(speaker)
                else:
                    speaker_set.append(speaker)
                    uniq_speaker = speaker_set.index(speaker)
                emotion = line[emo_idx]
                sess = line[sess_idx]

                if pre_sess == 'start' or sess == pre_sess:
                    session.append([uniq_speaker, utt, emotion])
                else:
                    self.session_dataset += split(session)
                    session = [[uniq_speaker, utt, emotion]]
                    speaker_set = []
                    emoSet.add(emotion)
                pre_sess = sess
        self.session_dataset += split(session)

        # self.emoList = sorted(emoSet) # 항상 같은 레이블 순서를 유지하기 위해
        self.emoList = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
        f.close()

    def __len__(self): # 기본적인 구성
        return len(self.session_dataset)

    def __getitem__(self, idx): # 기본적인 구성
        return self.session_dataset[idx]

    def padding(self, batch_input_token):
        # token 길이 512 넘으면 컷
        batch_token_ids, batch_attention_masks = batch_input_token['input_ids'], batch_input_token['attention_mask']
        trunc_batch_token_ids, trunc_batch_attention_masks = [], []
        for batch_token_id, batch_attention_mask in zip(batch_token_ids, batch_attention_masks):
            if len(batch_token_id) > self.tokenizer.model_max_length:
                trunc_batch_token_id = [batch_token_id[0]] + batch_token_id[1:][-self.tokenizer.model_max_length+1:]
                trunc_batch_attention_mask = [batch_attention_mask[0]] + batch_attention_mask[1:][-self.tokenizer.model_max_length+1:]
                trunc_batch_token_ids.append(trunc_batch_token_id)
                trunc_batch_attention_masks.append(trunc_batch_attention_mask)
            else:
                trunc_batch_token_ids.append(batch_token_id)
                trunc_batch_attention_masks.append(batch_attention_mask)

        # padding token으로 패딩하기
        max_length = max([len(x) for x in trunc_batch_token_ids])
        padding_tokens, padding_attention_masks = [], []
        for batch_token_id, batch_attention_mask in zip(trunc_batch_token_ids, trunc_batch_attention_masks):
            padding_tokens.append(batch_token_id + [self.tokenizer.pad_token_id for _ in range(max_length-len(batch_token_id))])
            padding_attention_masks.append(batch_attention_mask + [0 for _ in range(max_length-len(batch_token_id))]                                                        )
        return torch.tensor(padding_tokens), torch.tensor(padding_attention_masks)

    def collate_fn(self, sessions): # 배치를 위한 구성
        '''
            input:
                data: [(session1), (session2), ... ]
            return:
                batch_input_tokens_pad: (B, L) padded
                batch_labels: (B)
        '''
        ## [발화1, 발화2, ..., 발화8]
        # 발화1~발화7 context로 사용한다면 입력이 길어진다.
        # 발화1 같은 경우는 발화8에 별로 영향을 끼치지 않음.
        # 적절하게 context 길이를 조절해도 된다.
        # 3개로 정한다면, [발화5,발화6,발화7,발화8]

        batch_input, batch_labels = [], []
        batch_PM_input = []
        for session in sessions:
            input_str = self.tokenizer.cls_token

            # For PM
            current_speaker, current_utt, current_emotion = session[-1]
            PM_input = []
            for i, line in enumerate(session):
                speaker, utt, emotion = line
                input_str += " " + utt + self.tokenizer.sep_token
                if i < len(session)-1 and current_speaker == speaker:
                    PM_input.append(self.tokenizer.encode(utt, add_special_tokens=True, return_tensors='pt'))
                    # [cls_token, tokens, sep_token]

            # For CoM
            batch_input.append(input_str)
            batch_labels.append(self.emoList.index(emotion))
            batch_PM_input.append(PM_input)
        batch_input_token = self.tokenizer(batch_input, add_special_tokens=False)
        batch_padding_token, batch_padding_attention_mask = self.padding(batch_input_token)

        return batch_padding_token, batch_padding_attention_mask, batch_PM_input, torch.tensor(batch_labels)

In [18]:
from torch.utils.data import DataLoader
dev_dataset = data_loader('./MELD/data/MELD/dev_sent_emo.csv')
dev_dataloader = DataLoader(dev_dataset, batch_size=3, shuffle=False, num_workers=0, collate_fn=dev_dataset.collate_fn)

In [19]:
for i, data in enumerate(dev_dataloader):
    if i == 1:
        print(data[0].shape)
        batch_padding_token, batch_padding_attention_mask, batch_PM_input, batch_label = data
        print("batch_padding_token", batch_padding_token)
        print("batch_padding_attention_mask", batch_padding_attention_mask)
        print("batch_PM_input", batch_PM_input)
        print("batch_label", batch_label)
        break

torch.Size([3, 58])
batch_padding_token tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
           827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
             5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
           328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
            17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1],
        [    0,  1793,   328,  1793,  

In [20]:
from transformers import RobertaModel

model = RobertaModel.from_pretrained('roberta-base')

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

In [21]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_call_impl',
 '_convert_head_mask_to_5d',
 '_expand_inputs_for_generation',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_backward_hooks',
 '_get_decoder_start_token_id',
 '_get_logits_processor',
 '_get_logits_warper',
 '_get_name',
 '_get_pad_token_id',
 '_get_resized_embeddings',
 '_get_resized_lm_head',
 '_get_stopping_criteria',
 '_hook_rss_memory_post_forward',
 '_hook_rss_memory_pre_forward',
 '_init_sequence_length_for_generation',
 '_init_weights',
 '_is_full_backward_hook',
 '_keys_

In [22]:
model.train()
model.eval()

RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0): RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dropout): Drop

In [23]:
batch_padding_token.shape, batch_padding_attention_mask.shape

(torch.Size([3, 58]), torch.Size([3, 58]))

In [24]:
batch_padding_token, batch_padding_attention_mask

(tensor([[    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213,     7,     5,
            827,     6,   593,    84,  2349,     8,   847,   106,   160,    23,
              5,  1300,     4,     2,   370,    17,    27,   241,    10, 16333,
            328,     2,    83, 33130,     6,   313,     6,   122,    52,   351,
             17,    27,    90,    28,   827, 30489,   328,     2,     1,     1,
              1,     1,     1,     1,     1,     1,     1,     1],
         [    0,  1793,   328,  1793,     6,    52,   115,   213, 

In [25]:
# for CoM
batch_com_out = model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
print(batch_com_out.shape)
batch_com_final = batch_com_out[:,0,:] # CLS 토큰의 output 가져오기 위해
print(batch_com_final.shape)

torch.Size([3, 58, 768])
torch.Size([3, 768])


In [26]:
batch_PM_input

[[],
 [],
 [tensor([[    0,  1185,    17,    27,   241,    10, 16333,   328,     2]])]]

첫번째 batch와 두번째 batch는 비어있다. -> 앞에서 발화가 없었기 때문에!
세번째 batch에 대해서는 앞에서 나온 발화가 있었기 때문에, PM 입력이 있는 것!

In [27]:
model2 = RobertaModel.from_pretrained('roberta-base')

# GRU 세팅
import torch.nn as nn
hiddenDim = model2.config.hidden_size
zero = torch.empty(2, 1, hiddenDim)
h0 = torch.zeros_like(zero) # (num_layers * num_directions, batch, hidden_size)
speakerGRU = nn.GRU(hiddenDim, hiddenDim, 2, dropout=0.3) # (input, hidden, num_layer) (BERT_emb, BERT_emb, num_layer)

# GRU 통과 --> PM 결과
batch_pm_gru_final = []
for PM_inputs in batch_PM_input:
    if PM_inputs:
        pm_outs = []
        for PM_input in PM_inputs:
            pm_out = model2(PM_input)['last_hidden_state'][:,0,:] # CLS output
            pm_outs.append(pm_out)
        pm_outs = torch.cat(pm_outs, 0).unsqueeze(1) # (speaker_num, batch=1, hidden_dim)
        pm_gru_outs, _ = speakerGRU(pm_outs, h0) # (speaker_num, batch=1, hidden_dim)
        pm_gru_final = pm_gru_outs[-1,:,:] # (1, hidden_dim)
        batch_pm_gru_final.append(pm_gru_final)
    else:
        batch_pm_gru_final.append(torch.zeros(1, hiddenDim))
batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)

In [28]:
batch_pm_gru_final.shape

torch.Size([3, 768])

In [29]:
clsNum = len(dev_dataset.emoList)
W = nn.Linear(hiddenDim, clsNum)
final_output = W(batch_com_final + batch_pm_gru_final)
print(final_output.shape) # (B, C)

torch.Size([3, 7])


In [30]:
class ERC_model(nn.Module):
    def __init__(self, clsNum):
        super(ERC_model, self).__init__()
        self.com_model = RobertaModel.from_pretrained('roberta-base')
        self.pm_model = RobertaModel.from_pretrained('roberta-base')

        # GRU Settings
        self.hiddenDim = self.com_model.config.hidden_size
        zero = torch.empty(2, 1, self.hiddenDim)
        self.h0 = torch.zeros_like(zero) # (num_layers * num_directions, batch, hidden_size)
        self.speakerGRU = nn.GRU(self.hiddenDim, self.hiddenDim, 2, dropout=0.3) # (input, hidden, num_layer) (BERT_emb, BERT_emb, num_layer)

        # Score Matrix
        self.W = nn.Linear(self.hiddenDim, clsNum)
    def forward(self, batch_padding_token, batch_padding_attention_mask, batch_PM_input):
        # for CoM
        batch_com_out = self.com_model(input_ids=batch_padding_token, attention_mask=batch_padding_attention_mask)['last_hidden_state']
        batch_com_final = batch_com_out[:,0,:]

        # GRU 통과 --> PM 결과
        batch_pm_gru_final = []
        for PM_inputs in batch_PM_input:
            if PM_inputs:
                pm_outs = []
                for PM_input in PM_inputs:
                    pm_out = self.pm_model(PM_input)['last_hidden_state'][:,0,:]
                    pm_outs.append(pm_out)
                pm_outs = torch.cat(pm_outs, 0).unsqueeze(1) # (speaker_num, batch=1, hidden_dim)
                pm_gru_outs, _ = self.speakerGRU(pm_outs, self.h0) # (speaker_num, batch=1, hidden_dim)
                pm_gru_final = pm_gru_outs[-1,:,:] # (1, hidden_dim)
                batch_pm_gru_final.append(pm_gru_final)
            else:
                batch_pm_gru_final.append(torch.zeros(1, self.hiddenDim))
        batch_pm_gru_final = torch.cat(batch_pm_gru_final, 0)

        # score matrix
        final_output = self.W(batch_com_final + batch_pm_gru_final) # (B, C)

        return final_output

In [31]:
clsNum = len(dev_dataset.emoList)
model = ERC_model(clsNum)
result = model(batch_padding_token, batch_padding_attention_mask, batch_PM_input)
print(result.shape)

torch.Size([3, 7])


In [32]:
!touch model.py